In [ ]:

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ajyothi","key":"32dd5706f90d74efcd8decbfb8d1fe2c"}'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Make a directory for the Kaggle config
os.makedirs('/root/.kaggle', exist_ok=True)

# Move the kaggle.json file to the Kaggle config directory
!mv kaggle.json /root/.kaggle/

# Set the correct permissions
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d aryashah2k/indian-medicinal-leaves-dataset



Dataset URL: https://www.kaggle.com/datasets/aryashah2k/indian-medicinal-leaves-dataset
License(s): CC-BY-NC-SA-4.0
100% 9.00G/9.00G [06:17<00:00, 25.5MB/s]
100% 9.00G/9.00G [06:17<00:00, 25.6MB/s]


In [ ]:
import zipfile

# Unzip the downloaded file
with zipfile.ZipFile('indian-medicinal-leaves-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('indian_medicinal_leaves')



In [ ]:
import shutil

# Path to the directory you want to remove
dir_to_remove = '/content/indian_medicinal_leaves/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset'

# Remove the directory
shutil.rmtree(dir_to_remove)

print(f"Removed directory: {dir_to_remove}")


Removed directory: /content/indian_medicinal_leaves/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os


In [ ]:
# Define the base directory where data is stored
base_dir = '/content/indian_medicinal_leaves/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset'

# Check the number of classes (directories)
classes = os.listdir(base_dir)
print(f"Number of classes: {len(classes)}")


Number of classes: 80


In [ ]:
# Image dimensions
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Use 20% of data for validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Validation generator
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 5556 images belonging to 80 classes.
Found 1344 images belonging to 80 classes.


In [ ]:
# Load the InceptionV3 model without the top layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(len(classes), activation='softmax')(x)  # Output layer

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Adjust based on your compute power
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=validation_generator.samples // BATCH_SIZE
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 1164s 7s/step - accuracy: 0.1719 - loss: 3.8148 - val_accuracy: 0.3170 - val_loss: 2.8331
Epoch 2/10
  1/173 ━━━━━━━━━━━━━━━━━━━━ 11:53 4s/step - accuracy: 0.5312 - loss: 1.8057

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


173/173 ━━━━━━━━━━━━━━━━━━━━ 210s 1s/step - accuracy: 0.5312 - loss: 1.8057 - val_accuracy: 0.3013 - val_loss: 2.8849
Epoch 3/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 1128s 6s/step - accuracy: 0.5255 - loss: 1.7846 - val_accuracy: 0.3341 - val_loss: 2.6856
Epoch 4/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.5625 - loss: 1.6767 - val_accuracy: 0.3445 - val_loss: 2.6977
Epoch 5/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 1120s 6s/step - accuracy: 0.6239 - loss: 1.3531 - val_accuracy: 0.3966 - val_loss: 2.5287
Epoch 6/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.6875 - loss: 1.1501 - val_accuracy: 0.3988 - val_loss: 2.5552
Epoch 7/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 1130s 7s/step - accuracy: 0.6856 - loss: 1.0913 - val_accuracy: 0.4070 - val_loss: 2.5652
Epoch 8/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.5938 - loss: 1.5667 - val_accuracy: 0.4152 - val_loss: 2.4389
Epoch 9/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 1130s 7s/step - accuracy: 0.7230 - loss: 0.9644 - val_accuracy: 0

In [ ]:
# Evaluate on the validation set
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

42/42 ━━━━━━━━━━━━━━━━━━━━ 216s 5s/step - accuracy: 0.4545 - loss: 2.4148
Validation Accuracy: 0.45


In [ ]:
model.save('inception_medicinal_leaves.h5')


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Generate predictions for the validation set
y_true = validation_generator.classes
y_pred = np.argmax(model.predict(validation_generator), axis=-1)

# Get class indices and names
class_indices = validation_generator.class_indices
class_names = list(class_indices.keys())

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

# Extract accuracy (precision) for each species
species_accuracies = {species: report[species]['precision'] for species in class_names}
sorted_species = sorted(species_accuracies.items(), key=lambda x: x[1], reverse=True)

# Select top 15 species
top_species = sorted_species[:15]
top_species_names = [species for species, _ in top_species]

print("Top 15 species based on accuracy:")
for species, accuracy in top_species:
    print(f"{species}: {accuracy:.2f}")

In [ ]:
import shutil
import os

# Path to the original dataset
original_dataset_dir = '/content/indian_medicinal_leaves/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset'

# Path to the new dataset containing top 15 species
filtered_dataset_dir = '/content/top_15_species'

# Create directory for filtered dataset
if not os.path.exists(filtered_dataset_dir):
    os.makedirs(filtered_dataset_dir)

# Copy directories of top 15 species to the new dataset
for species in top_species_names:
    src = os.path.join(original_dataset_dir, species)
    dst = os.path.join(filtered_dataset_dir, species)
    shutil.copytree(src, dst)

print(f"Filtered dataset with top 15 species is available at: {filtered_dataset_dir}")


Filtered dataset with top 15 species is available at: /content/top_15_species
